In [1]:
from greatx.nn.models import GCN
from greatx.training import Trainer
from torch_geometric.datasets import Planetoid
from greatx.attack.untargeted import Metattack
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = Planetoid(root='.', name='Cora')
data = dataset[0]

In [3]:
model = GCN(dataset.num_features, dataset.num_classes)

In [4]:
trainer_before = Trainer(model, device=device)

In [5]:
trainer_before.fit(data, mask=data.train_mask)
trainer_before.evaluate(data, mask=data.test_mask)

Training...
100/100 [====================] - Total: 529.40ms - 5ms/step- loss: 0.036 - acc: 1
Evaluating...
1/1 [====================] - Total: 1.42ms - 1ms/step- loss: 0.641 - acc: 0.801


╒═════════╤═══════════╕
│ Names   │   Objects │
╞═════════╪═══════════╡
│ loss    │  0.641397 │
├─────────┼───────────┤
│ acc     │  0.801    │
╘═════════╧═══════════╛

In [6]:
attacker = Metattack(data, device=device)
attacker.setup_surrogate(trainer_before.model,
                         labeled_nodes=dataset.train_mask,
                         unlabeled_nodes=dataset.test_mask, lambda_=0.)
attacker.reset()
attacker.attack(0.05)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Metattack(
  device=cuda, seed=None,
  (surrogate): GCN(
    (conv): Sequential(
      (0): GCNConv(1433, 16)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GCNConv(16, 7)
    )
  )
)

In [7]:
logs = trainer_before.evaluate(attacker.data(), dataset.test_mask)
print(f"After evasion attack\n {logs}")

Evaluating...
1/1 [====================] - Total: 2.45ms - 2ms/step- loss: 0.666 - acc: 0.788
After evasion attack
 ╒═════════╤═══════════╕
│ Names   │   Objects │
╞═════════╪═══════════╡
│ loss    │  0.665707 │
├─────────┼───────────┤
│ acc     │  0.788    │
╘═════════╧═══════════╛


In [8]:
trainer_after = Trainer(GCN(dataset.num_features, dataset.num_classes), device=device)
trainer_after.fit(attacker.data(), mask=dataset.train_mask)
logs = trainer_after.evaluate(attacker.data(), dataset.test_mask)
print(f"After poisoning attack\n {logs}")

Training...
100/100 [====================] - Total: 268.66ms - 2ms/step- loss: 0.0489 - acc: 1
Evaluating...
1/1 [====================] - Total: 1.30ms - 1ms/step- loss: 1.55 - acc: 0.588
After poisoning attack
 ╒═════════╤═══════════╕
│ Names   │   Objects │
╞═════════╪═══════════╡
│ loss    │   1.54722 │
├─────────┼───────────┤
│ acc     │   0.588   │
╘═════════╧═══════════╛
